Code done by;        Aswathi Ajith

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml

import math
from pennylane.optimize import AdamOptimizer

In [2]:
print("new testing")

new testing


In [ ]:
import numpy as np

# Variables for the PDE
num_variables = int(input("Enter the number of variables for the PDE: "))
all_variable_orders = []
all_variable_coefficients = []
#
variable_names = []
variable_indices = []
variable_values = {}

# vraible names, their order of differentiation, and coefficients of those variables's derivatives
for i in range(num_variables):
    variable_name = input(f"Enter the name for variable {i + 1}: ")
    variable_names.append(variable_name)
    variable_indices.append(i)
    num_orders = int(input(f"Enter the number of derivatives for variable {variable_name}: "))
    orders = []
    coefficients = []

    print(f"Variable: {variable_name}")
    for j in range(num_orders):
        order = int(input(f"Enter the order for derivative {j + 1} with respect to {variable_name}: "))
        coefficient = input(f"Enter the coefficient for derivative {j + 1} with respect to {variable_name} (as a function of {variable_name}): ")
        orders.append(order)
        coefficients.append(coefficient)

    # To get the range values of variables
    low_value = float(input(f"Enter the low value for variable {variable_name}: "))
    high_value = float(input(f"Enter the high value for variable {variable_name}: "))
    # Discretizing the range with certain step size value
    step_size = float(input(f"Enter the step size for generating random numbers for variable {variable_name}: "))

    random_values = np.linspace(low_value, high_value, int((high_value - low_value) / step_size) + 1)
    print(f"Values for {variable_name}:")
    for value in random_values:
        print(value)

    all_variable_orders.append(orders)
    all_variable_coefficients.append(coefficients)
    variable_values[variable_name] = random_values.tolist()  

# Collect terms that are not derivatives in the PDE(e.g., u**2, sin(u), u, etc....)
non_derivative_terms = []
add_non_derivative_terms = input("Are there any non-derivative terms? type (yes/no): ").strip().lower()

while add_non_derivative_terms == "yes":
    term = input("Enter the non-derivative term : ")
    non_derivative_terms.append(term)
    add_non_derivative_terms = input("Is there any more non-derivative terms? (yes/no): ").strip().lower()

# To print all info;
print("Variable Names:", variable_names)
print("Variable Values:", variable_values)
print("Variable Orders:", all_variable_orders)
print("Variable Coefficients:", all_variable_coefficients)
print("Non-Derivative Terms:", non_derivative_terms)

# To construct the PDE expression
def construct_pde_expression(variable_names, all_variable_orders, all_variable_coefficients, non_derivative_terms):
    pde_expression = ""
    
    for i, (variable_name, orders, coefficients) in enumerate(zip(variable_names, all_variable_orders, all_variable_coefficients)):
        for order, coeff in zip(orders, coefficients):
            if "lambda" in coeff:  
                term = f"{coeff}*d^{order}u/d{variable_name}^{order}"
            else:
                term = f"{coeff}*d^{order}u/d{variable_name}^{order}"
            pde_expression += term + " + "

    
    for term in non_derivative_terms:
        pde_expression += term + " + "
    pde_expression = pde_expression[:-3] 
    return pde_expression


pde_expression = construct_pde_expression(variable_names, all_variable_orders, all_variable_coefficients, non_derivative_terms)
print("Constructed PDE:", pde_expression)




In [ ]:
#Check
variable_data = [(key, values) for key, values in variable_values.items()]
for variable, values in variable_data:
    print(f"{variable} = {values}",len(values))

In [ ]:
#construct the quantum circuit

cost_array =[]

chosen_variables = input("Enter the names of the variables to encode (comma-separated): ").split(',')
chosen_variables = [v.strip() for v in chosen_variables]


variable_dict = dict(variable_data)
for var in chosen_variables:
    if var not in variable_dict:
        raise ValueError(f"Variable '{var}' is not in the list of available variables.")


selected_values = [variable_dict[var] for var in chosen_variables]
selected_values = np.array(selected_values)
print(f"Selected values:\n{selected_values}")


num_qubits = len(chosen_variables)


dev = qml.device("default.qubit", wires=num_qubits)

# variational circuit
@qml.qnode(dev)
def variational_circuit(params, num_layers, variable_values):
    # Feature maps
    for i, value in enumerate(variable_values):
        qml.RY(2 * np.arccos(np.clip(value, -1, 1)), wires=i)

    # Parametrized variational circuits
    for layer in range(num_layers):
        for i in range(len(variable_values)):
            qml.RZ(params[layer, i, 0], wires=i)
            qml.RX(params[layer, i, 1], wires=i)
            # ansatz reduced --------------------
            
        for i in range(len(variable_values) - 1):
            qml.CNOT(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(len(variable_values))]

#  cost function
def cost_fn(params, num_layers, variable_data, chosen_variables):
    total_cost = 0
    num_iterations = len(variable_data[0][1])
  
    
    for i in range(num_iterations):
        selected_values = [variable_data[var_idx][1][i] for var_idx, var in enumerate(variable_data) if var[0] in chosen_variables]
        outputs = variational_circuit(params, num_layers, selected_values)
        
        cost = np.sum(outputs) #sigma(j)
        total_cost += cost #sigma(cost)
        print(f"Iteration {i+1}:")
        print(f"Variable Names: {chosen_variables}")
        print(f"Variable Values: {selected_values}")
        print(f"Cost: {cost}")
        print("outputs are",outputs)
        cost_array.append(cost)
    
    return total_cost,cost_array

# number of layers
num_layers = 6

# Initialize the parameters
np.random.seed(42)

params = np.random.uniform(-2*np.pi, 2*np.pi, size=(num_layers, num_qubits, 3))


# Calculate the cost
total_cost, cost_array = cost_fn(params, num_layers, variable_data, chosen_variables)


print(f"Total cost = {total_cost}")
print(f"The cost array is = {cost_array}")




In [15]:
# checked --------------- 

def compute_derivatives_for_variables(cost_fn, variable_names, variable_data, chosen_variables, shift, num_layers, params):
    derivatives = []
    variable_dict = dict(variable_data)
    
    for variable_to_differentiate in variable_names:
        num_orders = int(input(f"Enter the number of orders for {variable_to_differentiate}: "))
        if num_orders <= 0:
            print("Number of orders must be positive.")
            continue

        for j in range(1, num_orders + 1):
            variable_values_plus = {var: variable_dict[var][:] for var in variable_dict}
            variable_values_minus = {var: variable_dict[var][:] for var in variable_dict}

            # Apply the shift j times for the j-th order derivative
            for _ in range(j):
                variable_values_plus[variable_to_differentiate] = [a + shift for a in variable_values_plus[variable_to_differentiate]]
                variable_values_minus[variable_to_differentiate] = [a - shift for a in variable_values_minus[variable_to_differentiate]]

            # Convert variable data to the required format
            variable_data_plus = [(var, variable_values_plus[var]) for var in variable_dict]
            variable_data_minus = [(var, variable_values_minus[var]) for var in variable_dict]
            

            # Calculate the outputs (total cost only)
            output_plus, _ = cost_fn(params, num_layers, variable_data_plus, chosen_variables)
            output_minus, _ = cost_fn(params, num_layers, variable_data_minus, chosen_variables)
            print(output_minus)
            print(output_plus)

            # Calculate the derivative using total cost
            derivative = (output_plus - output_minus) / (2)
            derivatives.append((variable_to_differentiate, j, derivative))

    return derivatives


shift = 0.01  
derivatives = compute_derivatives_for_variables(cost_fn, chosen_variables, variable_data, chosen_variables, shift, num_layers, params)
print(derivatives)

In [ ]:
for variable, order, derivative in derivatives:
    print(f"Variable: {variable}, Order: {order}, Derivative: {derivative}")

In [ ]:
# To calculate the PDE value from the fucntion and the derivative values 

def calculate_PDE_from_derivatives(derivatives, variable_names, total_cost):
    print("Derivatives:")
    for derivative in derivatives:
        print(f"Variable: {derivative[0]}, Order: {derivative[1]}, Value: {derivative[2]}")

    num_derivative_terms = int(input("Enter the number of derivative terms in the PDE: "))
    num_non_derivative_terms = int(input("Enter the number of non-derivative terms in the PDE: "))
    
    PDE_value = 0
    # To get the details of the derivative terms
    for _ in range(num_derivative_terms):
        variable = input(f"Enter variable for the derivative term from the list {variable_names}: ")
        order = int(input(f"Enter the order of the derivative for {variable}: "))
        coefficient_input = input(f"Enter the coefficient for the derivative term with {variable} of order {order} (can be a variable, 'total_cost', or numerical value): ")
        
        # To check if the coefficient of the derivative is the mutliplied by fn value(as in non linear PDE) or just a numerical value
        if coefficient_input in variable_names:
            coefficient = float(input(f"Enter value for the variable {coefficient_input}: "))
            
        elif coefficient_input == 'total_cost':
            coefficient = total_cost
        else:
            coefficient = float(coefficient_input)
        selected_derivative = None
        for derivative in derivatives:
            if derivative[0] == variable and derivative[1] == order:
                selected_derivative = derivative[2]
                break
        if selected_derivative is not None:
            PDE_value += coefficient * selected_derivative
        else:
            return f"Derivative value not found for variable {variable} with order {order}."
    
    # To get the non derivative terms in the PDE 

    for _ in range(num_non_derivative_terms):
        coefficient_input = input("Enter the coefficient for the non-derivative term : ") # this can also be a numerical coefficient or the fucntion value itslef
        
        if coefficient_input in variable_names:
            coefficient = float(input(f"Enter value for the variable {coefficient_input}: "))
        elif coefficient_input == 'total_cost':
            coefficient = total_cost
        else:
            coefficient = float(coefficient_input)
        
        non_derivative_expr = input("Enter the expression for the non-derivative term : ")
        variable_used = input("Which variable is used in the expression? Enter the variable name or 'total_cost': ")
        
        variable_values = {var: 0 for var in variable_names}
        if variable_used in variable_names:
            variable_values[variable_used] = float(input(f"Enter value for the variable {variable_used}: "))
        elif variable_used == 'total_cost':
            variable_values['total_cost'] = total_cost
        
        non_derivative_value = eval(non_derivative_expr, {"__builtins__": None}, variable_values)
        PDE_value += coefficient * non_derivative_value
    
    return PDE_value

# Final PDE value
PDE_value = calculate_PDE_from_derivatives(derivatives, variable_names, total_cost)
print(f"PDE value: {PDE_value}")


In [ ]:

global initial_condition_values, boundary_condition_values
initial_condition_values = None
boundary_condition_values = None


# To get the initial condition
def handle_initial_condition():
    global initial_condition_values
    print("List of variables:", variable_names)
    variables_in_condition = input("Enter the variables involved in the initial condition separated by commas: ").split(',')
    
    valid_variables = [var.strip() for var in variables_in_condition if var.strip() in variable_names]
    
    if len(valid_variables) == len(variables_in_condition):
        initial_condition_expr = input("Enter the initial condition expression at t=0: ")
        
        variable_values_combinations = [variable_values[var] for var in valid_variables]
        
        try:
            print("Initial Condition Expression:", initial_condition_expr)
            
            initial_condition_values = []
            for values in zip(*variable_values_combinations):
                eval_context = {var: val for var, val in zip(valid_variables, values)}
                eval_context.update({'np': np, 'math': math})
                initial_condition = eval(initial_condition_expr, eval_context)
                initial_condition_values.append(initial_condition)
            
            initial_condition_values = np.array(initial_condition_values)
            return initial_condition_values
        
        except Exception as e:
            print("Invalid expression. Please enter a valid mathematical expression.")
            print(e)
            return None
    else:
        print("Invalid variable selection. Please choose variables from the provided list.")
        return None



def handle_boundary_conditions():
    global boundary_condition_values
    boundary_condition_values = {}

    num_boundary_conditions = int(input("Enter number of boundary conditions: "))

    for _ in range(num_boundary_conditions):
        # For type of boundary condition (Neumann, Dirichlet, or Periodic)
        condition_type = input("Enter boundary condition type (Neumann, Dirichlet, or Periodic): ").strip().lower()

        if condition_type not in ['neumann', 'dirichlet', 'periodic']:
            print("Invalid boundary condition type. Please enter 'Neumann', 'Dirichlet', or 'Periodic'.")
            continue

        variables_in_condition = input("Enter variables involved in the boundary condition (comma-separated): ").split(',')
        valid_variables = [var.strip() for var in variables_in_condition if var.strip() in variable_names]

        if len(valid_variables) != len(variables_in_condition):
            print("Invalid variable selection. Please choose variables from the provided list.")
            continue

        if condition_type == 'dirichlet':
            for var in valid_variables:
                points = input(f"Enter the points for {var} where the Dirichlet condition should be applied (comma-separated): ").split(',')
                points = [point.strip() for point in points]
                
                lower_value = input(f"Enter Dirichlet boundary condition at lower {var} (value or expression): ")
                upper_value = input(f"Enter Dirichlet boundary condition at upper {var} (value or expression): ")

                boundary_condition_values[(var, 'dirichlet')] = {
                    'points': points,
                    'lower_value': lower_value,
                    'upper_value': upper_value
                }

        elif condition_type == 'neumann':
            for var in valid_variables:
                points = input(f"Enter the points for {var} where the Neumann condition should be applied (comma-separated): ").split(',')
                points = [point.strip() for point in points]
                
                derivative_direction = input(f"Enter derivative direction for Neumann condition with respect to {var} : ").strip().lower()

                if derivative_direction not in variable_names:
                    print("Invalid derivative direction")
                    continue

                neumann_condition = input(f"Enter Neumann boundary condition for {var} ({derivative_direction}-derivative) (value or expression): ")

                boundary_condition_values[(var, 'neumann')] = {
                    'points': points,
                    'derivative_direction': derivative_direction,
                    'neumann_condition': neumann_condition
                }

        elif condition_type == 'periodic':
            for var in valid_variables:
                points = input(f"Enter the points for {var} where the function value should be the same (comma-separated): ").split(',')
                points = [point.strip() for point in points]
                
                boundary_condition_values[(var, 'periodic')] = {
                    'points': points,
                    'type': 'periodic'
                }

    return boundary_condition_values
# Function call to handle initial and boundary conditions
condition_type = input("Enter the problem type (initial/boundary/both): ").strip().lower()

if condition_type == 'initial':
    initial_condition_values = handle_initial_condition()
    print("Initial Condition Values:", initial_condition_values)
elif condition_type == 'boundary':
    boundary_condition_values = handle_boundary_conditions()
    print("Boundary Condition Values:", boundary_condition_values)
elif condition_type == 'both':
    initial_condition_values = handle_initial_condition()
    boundary_condition_values = handle_boundary_conditions()
    print("Initial Condition Values:", initial_condition_values)
    print("Boundary Condition Values:", boundary_condition_values)
else:
    print("Invalid input. Please enter 'initial', 'boundary', or 'both'.")




In [ ]:
print(variable_data[1][1])
print(initial_condition_values)
plt.plot(variable_data[1][1], initial_condition_values, label='Cost vs. Variable Values')
plt.xlabel(f'Values of {chosen_variables[0]}')
plt.ylabel('Cost')
plt.title('Cost Array vs. Variable Values')
plt.legend()
plt.show()

In [ ]:
def calculate_boundary_loss(boundary_condition_values, cost_array, weight, variable_dict):
    boundary_loss_lower = []
    boundary_loss_upper = []
    for key, value in boundary_condition_values.items():
        print(f"Processing key: {key}")  
        if key[1] == 'dirichlet':
            var = key[0]
            points = value['points']
            lower_value = float(value.get('lower_value', None))
            upper_value = float(value.get('upper_value', None))
            print(f"Dirichlet condition for {var}: lower_value = {lower_value}, upper_value = {upper_value}")  
            for point in points:
                point_value = float(point)
                if point_value in variable_dict[var]:
                    index = variable_dict[var].index(point_value)
                    predicted_cost = cost_array[index]
                    print(f"Point {point_value}: predicted_cost = {predicted_cost}")  

                    if point_value == min(variable_dict[var]):
                        loss = weight * (lower_value - predicted_cost) ** 2
                        boundary_loss_lower.append(loss)
                        print(f"Lower boundary loss at point {point_value}: {loss}")  
                    elif point_value == max(variable_dict[var]):
                        loss = weight * (upper_value - predicted_cost) ** 2
                        boundary_loss_upper.append(loss)
                        print(f"Upper boundary loss at point {point_value}: {loss}")  
        elif key[1] == 'neumann':
            var = key[0]
            points = value['points']
            neumann_condition = float(value['neumann_condition'])
            print(f"Neumann condition for {var}: neumann_condition = {neumann_condition}")  
            for point in points:
                point_value = float(point)
                if point_value in variable_dict[var]:
                    index = variable_dict[var].index(point_value)
                    predicted_cost = cost_array[index]
                    loss = weight * (neumann_condition - predicted_cost) ** 2
                    boundary_loss_lower.append(loss)
                    boundary_loss_upper.append(loss)
                    print(f"Neumann boundary loss at point {point_value}: {loss}")  
    return boundary_loss_lower[0], boundary_loss_upper[0]
weight = 1  
boundary_loss_lower, boundary_loss_upper = calculate_boundary_loss(boundary_condition_values, cost_array, weight, variable_dict)
print(f"Boundary loss lower = {boundary_loss_lower}")
print(f"Boundary loss upper = {boundary_loss_upper}")


In [ ]:
import numpy as np
# Function to construct initial condition data
def construct_initial_condition_data(variable_data):
    variable_data_initial = {}
    # Ask user which variables are involved in the initial condition
    initial_condition_vars_input = input("Enter the variables involved in the initial condition (comma-separated): ")
    initial_condition_vars = [var.strip() for var in initial_condition_vars_input.split(',')]
    for var_name, var_values in variable_data:
        if var_name == 't':
            # For 't', set all values to 0.0
            variable_data_initial[var_name] = [0.0] * len(var_values)
        elif var_name in initial_condition_vars:
            # For other variables involved in initial condition, retain original values
            variable_data_initial[var_name] = var_values
        else:
            # For variables not involved in initial condition, retain original values
            variable_data_initial[var_name] = var_values
    return variable_data_initial
variable_data_initial = construct_initial_condition_data(variable_data)
print("Initial Condition Data:")
print(variable_data_initial)




In [ ]:

def cost_fn_initial(params, num_layers, variable_data_initial):
    total_cost = 0
    cost_array = []
    num_iterations = len(next(iter(variable_data_initial.values())))  
    
    num_qubits = len(variable_data_initial)
    dev = qml.device("default.qubit", wires=num_qubits)

    # variational circuit
    @qml.qnode(dev)
    def variational_circuit(params, num_layers, variable_values):
        # Feature maps
        for i, value in enumerate(variable_values):
            qml.RY(2 * np.arccos((value)), wires=i)
        # Parametrized variational circuits
        for layer in range(num_layers):
            for i in range(len(variable_values)):
                qml.RZ(params[layer, i, 0], wires=i)
                qml.RX(params[layer, i, 1], wires=i)
                # ansatz reduced --------------------  
            for i in range(len(variable_values) - 1):
                qml.CNOT(wires=[i, i + 1])
        return [qml.expval(qml.PauliZ(i)) for i in range(len(variable_values))]
    for i in range(num_iterations):
        selected_values = [variable_data_initial[var][i] for var in variable_data_initial]
        print(selected_values)
        outputs = variational_circuit(params, num_layers, selected_values)
        cost = np.sum(outputs)
        total_cost += cost
        print(f"Iteration {i+1}:")
        print(f"Variable Names: {list(variable_data_initial.keys())}")
        print(f"Variable Values: {selected_values}")
        print(f"Cost: {cost:.4f}")  # Adjusted to format the cost to four decimal places
        cost_array.append(cost)
    return total_cost, cost_array
# Function to calculate the IC loss
def cost_fn_ic(cost_array_initial, initial_condition_values):
    ic_loss_values = []
    weight = 1
    for i in range(len(cost_array_initial)):
        # print(initial_condition_values[i])
        ic_loss_values.append(weight * ((cost_array_initial[i] - initial_condition_values[i]) ** 2))
    return ic_loss_values
# Validate chosen variables
chosen_variables = input("Enter the names of the variables to encode (comma-separated): ").split(',')
chosen_variables = [v.strip() for v in chosen_variables]
variable_dict = dict(variable_data)
for var in chosen_variables:
    if var not in variable_dict:
        raise ValueError(f"Variable '{var}' is not in the list of available variables.")
# Calculate the total cost using the initial condition data
total_cost_initial, cost_array_initial = cost_fn_initial(params, num_layers, variable_data_initial)
print(f"Total cost = {total_cost_initial}")
print(f"Initial cost array = {cost_array_initial}")
print(f"Length of cost_array_initial: {len(cost_array_initial)}")
# Calculate the IC loss values
ic_loss_values = cost_fn_ic(cost_array_initial, initial_condition_values)
# Print the mean of IC loss values
print(ic_loss_values)
ic_loss = np.mean(ic_loss_values)
print("Mean IC loss:", ic_loss)
plt.figure()
print(variable_data[1][1])
plt.plot(variable_data[1][1], cost_array_initial, label='u(x, 0)', marker='o', linestyle='-', color='b')
plt.xlabel(f'Values of {chosen_variables[0]}')
plt.ylabel('Cost')
plt.title('Cost Array vs. Variable Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
pde_loss = ((PDE_value - 0) ** 2)/101
print(pde_loss)

In [ ]:

def calculate_loss(PDE_value, ic_loss, boundary_loss_lower,boundary_loss_upper):
    total_loss = pde_loss +  ic_loss +  boundary_loss_lower + boundary_loss_upper
    return total_loss

total_loss = calculate_loss(PDE_value, ic_loss, boundary_loss_lower,boundary_loss_upper)
print('Total loss is', total_loss)




In [ ]:



alpha = 0.03  
ic_weight = 1
bc_weight = 1
num_iterations = 30

loss_values = []
iterations = []
params_new = []



opt = qml.GradientDescentOptimizer(alpha)

for n in range(num_iterations):
    try:
        # Compute derivatives
        derivatives = compute_derivatives_for_variables(cost_fn, variable_names, variable_data, chosen_variables, shift, num_layers, params)
        if derivatives is None:
            print("Error: Derivatives are None")
            break

        numerical_derivatives = [derivative[2] for derivative in derivatives]

        print(f"Gradients: {numerical_derivatives}")


        gradient = np.array([np.sum(derivative) for derivative in numerical_derivatives])


        gradient = np.clip(gradient, -1.0, 1.0)

        # Update the parameters
        params = params - alpha * gradient

      
        PDE_value = calculate_PDE_from_derivatives(derivatives, chosen_variables, total_cost_initial)
        if PDE_value is None:
            print("Error: PDE value is None")
            break

        # for the initial conditions
        total_cost_initial, cost_array_initial = cost_fn_initial(params, num_layers, variable_data_initial)
        ic_loss_values = cost_fn_ic(cost_array_initial, initial_condition_values)
        ic_loss = np.mean(ic_loss_values)

        print("Mean IC loss:", ic_loss)

        # for boundary condition loss
        boundary_loss_lower, boundary_loss_upper = calculate_boundary_loss(boundary_condition_values, cost_array, weight, variable_dict)
        

        pde_loss = np.mean((PDE_value - 0) ** 2)
       
        loss = pde_loss +  ic_loss +  boundary_loss_lower + boundary_loss_upper


        # Store the loss value and iteration number
        loss_values.append(loss)
        iterations.append(n)
        params_new.append(params.flatten())

        print(f"Iteration {n+1}, Loss: {loss}, First parameter: {params.flatten()[0]}")

    except Exception as e:
        print(f"Error: {e}")
        break

# Plot the loss values vs. iteration number
plt.figure(figsize=(10, 6))
plt.plot(iterations, loss_values, marker='o', linestyle='-', color='b')
plt.title('Loss values vs. Iteration Number')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid(True)
plt.show()